In [ ]:
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow
import pandas as pd

# Build Strategy and model:

For simplicity sake I am using a pretrained ResNet152 model.

You will get faster training times using a smaller model but typically see better accuracy with larger models. 

The point of this exercise is to train a model using multiple GPU's on the google cloud. So I made sure to use a model structure where this was necessary to see improved results.


### Mirror Strategy:

The TensorFlow Distributed library comes with several distribution options. 

The Mirrored strategy allows for training on multiple GPU's the gradients are then calculated through all-reduce.


See the TensorFlow Documentation for more information.


https://www.tensorflow.org/guide/distributed_training



In [ ]:
mirrored_strategy = tensorflow.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2","/gpu:3"])


####

# When working with TensorFlow Distributed the model needs to be built inside the scope.
# Since this notebook was written for the use of 4x NVIDIA P100's I have specified each GPU by it's label
# Inside the scope below I am importing a pretrained ResNet152 Architecture and replacing it's final layer
# I am then setting all of the layers to trainable. (Again there are better ways to practice this I did so for simplicity).
# After the model is built you compile it. Since the DataSet has four categoried I am using Categorical Crossentropy and Adam

####

with mirrored_strategy.scope():
    resnet_base = ResNet152()
    layers = [l for l in resnet_base.layers]
    new_out = Dense(4, activation='softmax')(layers[-2].output)
    resmodel = Model(inputs= resnet_base.input, outputs=new_out)
    
    for layer in resmodel.layers:
        layer.trainable=True
    resmodel.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
        

## DataGens:
Since we have our Data stored in DataFrames I am creating a Train and Validation Generator. Only a Train and Test set were provided so I will start by splitting off a portion for Validation Data. 

When working with image Data (Or most data really..) it is considered a best practice to rescale to between 0 and 1.

I am also creating horizontal and vertical flips as well as providing random zooms between 80% and 120% in order to generate more data. The given 1800 images is a considerably small dataset.

With The Validation Data I am only rescaling since that is the only transformation I will be performing on test (or production data)


In [ ]:
datagen = ImageDataGenerator(zoom_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             rescale=1./255)

train = pd.read_csv('./Train_Test/Train.csv').drop('Unnamed: 0', axis=1)

val = train[round(len(train)*0.95):]
train = train[0:round(len(train)*0.95)]

train_gen = datagen.flow_from_dataframe(train, x_col='path', y_col=['healthy', 'multiple_diseases', 'rust', 'scab'], 
                                        class_mode='raw', target_size=(224, 224), batch_size=64)
val_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(val, x_col='path', y_col=['healthy', 'multiple_diseases', 'rust', 'scab'], 
                                                                class_mode='raw', target_size=(224, 224), batch_size=64)

## Callback and Train:

When fitting deep networks the learning rate will need to be adjusted otherwise you hover around the minimum without ever converging. 

I am using Keras ReduceLROnPlateau to handle this for me. 

Basically, after the validation score doesn't improve for 5 epochs the training rate will lower so more "fine" tuning can be done

Afterwards I am fitting for 150 Epochs. (This number is mostly arbitrary. Another Callback could be assigned to perform Early Stopping with patience of say "20") But I am just running this as a preliminary.

In [ ]:
from keras.callbacks import ReduceLROnPlateau

rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)


resmodel.fit(train_gen, validation_data=val_gen, epochs=150, callbacks=[rlrop])

## Test:

After the model is finished training we run predictions on the test DataSet. 

The main differences here are to set ```Shuffle= False``` Otherwise the data won't be in the same order as we passed it in and, ```class_mode=None```. 

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(dataframe= pd.read_csv('./Train_Test/Test.csv').drop('Unnamed: 0', axis=1),
                                                                  x_col="path",
                                                                  y_col=None,
                                                                  batch_size=64,
                                                                  shuffle=False,
                                                                  class_mode=None,
                                                                  target_size=(224,224))

preds = resmodel.predict(test_gen)

## Submission:

Building the submission (Remember to set your index to False when finishing the csv)

The rest of this section is just some simple Pandas and Numpy to structure the data.

Rememeber if you want to look at what any of these objects look like just create a new cell and run it with the object (example: df) within it and it will print below the cell.

In [ ]:
df = pd.DataFrame(preds)

In [ ]:
test = pd.read_csv('./Train_Test/Test.csv').drop('Unnamed: 0', axis=1)
df

In [ ]:
subs_pre = test.join(df)

In [ ]:
subs_pre

In [ ]:
subs_pre.drop('path', inplace=True, axis=1)

In [ ]:
subs_named = subs_pre.rename(columns={0:'healthy', 1:'multiple_diseases', 2:'rust', 3:'scab'})

In [ ]:
import numpy as np
subs_named['healthy'] = np.where(subs_named['healthy'] > 0.75, 1., 0.0)
subs_named['multiple_diseases'] = np.where(subs_named['multiple_diseases'] > 0.75, 1., 0.0)
subs_named['rust'] = np.where(subs_named['rust'] > 0.75, 1., 0.0)
subs_named['scab'] = np.where(subs_named['scab'] > 0.75, 1., 0.0)

In [ ]:
subs_named.to_csv('./first_submission.csv', index=False)